## Tools

In [ ]:
!pip install Together

In [ ]:
import requests
import re
import json
from typing import List, Dict
from google.colab import userdata
from together import Together

In [ ]:
client = Together(api_key=userdata.get('TOGETHER_API_KEY'))

In [ ]:
# Topic Breakdown Tool
def topic_breakdown(topic):
    prompt = f"Break down the following research topic into smaller, more focused subtopics:\n\n{topic}"
    response_text = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            messages=[
                  {"role": "user", "content": topic}
                ],
            max_tokens=256,
            temperature=0.7,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["<|eot_id|>"],
            stream=False
        ).choices[0].message.content
    subtopics = re.findall(r'\*\*(.*?)\*\*', response_text)
    return [subtopic.strip() for subtopic in subtopics if subtopic.strip()]

In [ ]:
# Query Expansion Tool
def query_expansion(subtopics):
    expanded_queries = []

    for subtopic in subtopics:
        prompt = f"Generate related keywords, synonyms, and phrases for the following subtopic:\n\n{subtopic}"
        response_text = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=256,
            temperature=0.7,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["<|eot_id|>"],
            stream=False
        ).choices[0].message.content

        keywords = re.findall(r'\*\*Related Keywords:\*\*\n\n(.*?)(\n\n|\*\*)', response_text, re.DOTALL)
        synonyms = re.findall(r'\*\*Synonyms:\*\*\n\n(.*?)(\n\n|\*\*)', response_text, re.DOTALL)
        phrases = re.findall(r'\*\*Phrases:\*\*\n\n(.*?)(\n\n|\*\*)', response_text, re.DOTALL)

        keywords_list = re.findall(r'\d+\.\s(.*?)\n', keywords[0][0]) if keywords else []
        synonyms_list = re.findall(r'\d+\.\s(.*?)\n', synonyms[0][0]) if synonyms else []
        phrases_list = re.findall(r'\d+\.\s"(.*?)"', phrases[0][0]) if phrases else []

        expanded_queries += keywords_list + synonyms_list + phrases_list

    expanded_queries = [query.strip() for query in expanded_queries]
    expanded_queries = list(set(expanded_queries))
    return expanded_queries

In [ ]:
# Search Tool with You.com
def search(queries):
    headers = {"X-API-Key": userdata.get('YOUR_API_KEY')}
    search_results = []

    for query in queries:
        params = {"query": query}
        response = requests.get(
            f"https://api.ydc-index.io/search",
            params=params,
            headers=headers,
        ).json()
        search_results.extend(hit['title'] + " - " + hit['url'] for hit in response['hits'] if response['hits'])

    return search_results

In [ ]:
# Critique Tool
def critique_summary(summary):
    prompt = f"Please critique the following summary and provide suggestions for improvement: {summary}"
    critique = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=256,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"],
        stream=False
    ).choices[0].message.content

    return critique

In [ ]:
# Summarizer Tool
def summarize_content(content):
    prompt = f"Please summarize the following content: {content}"
    summary = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=256,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"],
        stream=False
    ).choices[0].message.content

    return summary

In [ ]:
# Improve Summary Tool
def improve_summary(summary, critique):
    prompt = f"""
    Initial Summary:
    {summary}

    Critique and Suggestions:
    {critique}

    Based on the above critique and suggestions, improve the initial summary to address the points mentioned.
    """

    improved_summary = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=256,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"],
        stream=False
    ).choices[0].message.content

    return improved_summary


## Workflow

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "topic_breakdown",
            "description": "Break down a research topic into smaller subtopics",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The research topic to break down"
                    }
                },
                "required": ["topic"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_expansion",
            "description": "Generate related keywords, synonyms, and phrases for subtopics",
            "parameters": {
                "type": "object",
                "properties": {
                    "subtopics": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "A list of subtopics"
                    }
                },
                "required": ["subtopics"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Perform a search query",
            "parameters": {
                "type": "object",
                "properties": {
                    "queries": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "A list of search queries"
                    }
                },
                "required": ["queries"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_content",
            "description": "Generate a summary of the provided content",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "The content to summarize"
                    }
                },
                "required": ["content"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "critique_summary",
            "description": "Provide a critique of the generated summary",
            "parameters": {
                "type": "object",
                "properties": {
                    "summary": {
                        "type": "string",
                        "description": "The summary to critique"
                    }
                },
                "required": ["summary"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "improve_summary",
            "description": "Improve the generated summary based on critique",
            "parameters": {
                "type": "object",
                "properties": {
                    "summary": {
                        "type": "string",
                        "description": "The summary to improve"
                    },
                    "critique": {
                        "type": "string",
                        "description": "The critique of the summary"
                    }
                },
                "required": ["summary", "critique"]
            }
        }
    }
]


In [ ]:
# build tool_prompt for later use

tool_descriptions = []
for tool in tools:
    tool_descriptions.append(f"Use the function '{tool['function']['name']}' to '{tool['function']['description']}':\n{json.dumps(tool['function'])}\n")

tool_prompt = """
You have access to the following functions:

""" + "\n".join(tool_descriptions) + """
If you choose to call a function ONLY reply in the following format with no prefix or suffix:

<function=example_function_name>{"example_name": "example_value"}</function>

Reminder:
- Function calls MUST follow the specified format, start with <function= and end with </function>
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- If there is no function call available, answer the question like normal with your current knowledge and do not tell the user about function calls

"""
print(tool_prompt)


You have access to the following functions:

Use the function 'topic_breakdown' to 'Break down a research topic into smaller subtopics':
{"name": "topic_breakdown", "description": "Break down a research topic into smaller subtopics", "parameters": {"type": "object", "properties": {"topic": {"type": "string", "description": "The research topic to break down"}}, "required": ["topic"]}}

Use the function 'query_expansion' to 'Generate related keywords, synonyms, and phrases for subtopics':
{"name": "query_expansion", "description": "Generate related keywords, synonyms, and phrases for subtopics", "parameters": {"type": "object", "properties": {"subtopics": {"type": "array", "items": {"type": "string"}, "description": "A list of subtopics"}}, "required": ["subtopics"]}}

Use the function 'search' to 'Perform a search query':
{"name": "search", "description": "Perform a search query", "parameters": {"type": "object", "properties": {"queries": {"type": "array", "items": {"type": "string"}, 

In [ ]:
# quick test for ayuomatic function selection
messages = [
    {
        "role": "system",
        "content": tool_prompt,
    },
    {
        "role": "user",
        "content": "Perform a search query - ['Intelligent Tutoring Systems (ITS)', 'Personalized Learning', 'Automated Grading', 'Natural Language Processing (NLP)', 'Virtual Learning Environments (VLEs)', 'Predictive Analytics', 'Content Recommendation']['Intelligent Tutoring Systems (ITS)', 'Personalized Learning', 'Automated Grading', 'Natural Language Processing (NLP)', 'Virtual Learning Environments (VLEs)', 'Predictive Analytics', 'Content Recommendation']",
    },
]

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    messages=messages,
    max_tokens=1024,
    temperature=0,
)

messages.append(response.choices[0].message)
print(response.choices[0].message.content)

<function=search>{"queries": "['Intelligent Tutoring Systems (ITS)', 'Personalized Learning', 'Automated Grading', 'Natural Language Processing (NLP)', 'Virtual Learning Environments (VLEs)', 'Predictive Analytics', 'Content Recommendation']"}</function>


In [ ]:
# define the function parse tool
def parse_tool_response(response: str):
    function_regex = r"<function=(\w+)>"
    match = re.search(function_regex, response)

    if match:
        function_name = match.group(1)
        return {
            "function": function_name,
        }
    return None

parsed_response = parse_tool_response(response.choices[0].message.content)
print(parsed_response)

{'function': 'search'}


In [ ]:
# Available functions
available_functions = {
    "topic_breakdown": topic_breakdown,
    "query_expansion": query_expansion,
    "search": search,
    "summarize_content": summarize_content,
    "critique_summary": critique_summary,
    "improve_summary": improve_summary
}

# target steps
steps = [
    "Break down the research topic",
    "Generate related keywords, synonyms, and phrases for subtopics",
    "Perform a search query",
    "Please summarize the following content",
    "Provide a critique of the generated summary",
    "Improve the generated summary based on critique"
]

In [ ]:
cache = {
        'subtopics': [],
        'expanded_queries': [],
        'search_results': [],
        'summary': "",
        'critique': "",
        'improved_summary': ""
}

# Workflow
def research_agent(prompt):
    global cache

    for step in steps:
        print(f"Step: {step}")

        messages = [
            {
                "role": "system",
                "content": tool_prompt,
            },
            {
                "role": "user",
                "content": f"{step} - {prompt}",
            },
        ]

        # automatic decide which tool to use
        response = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            messages=messages,
            max_tokens=1024,
            temperature=0,
        )

        parsed_response = parse_tool_response(response.choices[0].message.content)
        # print(f"Parsed Response: {parsed_response}")

        if parsed_response:
            function_name = parsed_response["function"]

            if function_name in available_functions:
                print(f"Function Called: {function_name}")

                function_to_call = available_functions[function_name]
                if function_name == 'topic_breakdown':
                    result = function_to_call(prompt)
                    cache['subtopics'] = result
                    result = json.dumps(result)

                if function_name == 'query_expansion':
                    result = function_to_call(cache['subtopics'])
                    cache['expanded_queries'] = result
                    result = json.dumps(result)

                if function_name == 'search':
                    result = function_to_call(cache['expanded_queries'])
                    cache['search_results'].extend(result)
                    result = "\n".join(result)

                if function_name == 'summarize_content':
                    result = function_to_call(result)
                    cache['summary'] = result
                    result = json.dumps(result)

                if function_name == 'critique_summary':
                    result = function_to_call(cache['summary'])
                    cache['critique'] = result
                    result = json.dumps(result)

                if function_name == 'improve_summary':
                    result = function_to_call(cache['summary'], cache['critique'])
                    cache['improved_summary'] = result
                    result = json.dumps(result)

                prompt = result
                print(f"Result: {prompt}\n\n")

        else:
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {
                    "role": "user",
                    "content": f"{step} - {prompt}",
                },
            ]

            response = client.chat.completions.create(
                model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
                messages=messages,
                max_tokens=1024,
                temperature=0,
            )

            prompt = response.choices[0].message.content
            print(f"Result: {prompt}\n\n")

    return cache['improved_summary']


In [ ]:
# Execute the research agent
topic = "Applications of Artificial Intelligence in Education"
result = research_agent(topic)
print(result)


Step: Break down the research topic
Function Called: topic_breakdown
Result: ["Personalized Learning", "Intelligent Tutoring Systems", "Automated Grading", "Natural Language Processing (NLP)", "Predictive Analytics", "Content Creation", "Virtual Learning Environments"]


Step: Generate related keywords, synonyms, and phrases for subtopics
Function Called: query_expansion
Result: ["Virtual education platforms", "Analysis", "Simulation", "Content production", "Forecasting", "Insight Generation", "Learning management systems (LMS)", "Content building", "Computational Linguistics", "Sentiment Analysis = Opinion Mining, Emotion Detection, Attitude Analysis", "Machine Learning", "Deep Learning", "Digital learning spaces", "Text Analysis = Text Mining, Text Analytics, Text Information Extraction", "Student-driven learning", "Automated scoring", "Streamlining grading with AI", "Using technology to grade assignments", "Prognostication", "Human-Computer Interaction (HCI)", "Learning profiles", "

## Format Result

In [ ]:
def extract_title_and_link(result):
    match = re.match(r'(.*?) - (https?://\S+)', result)
    if match:
        return match.groups()
    return None, None

In [ ]:
from IPython.display import display, Markdown

references = []
for i, result in enumerate(cache['search_results']):
    title, link = extract_title_and_link(result)
    if title and link:
        references.append(f"{i+1}. [{title}]({link})")
formatted_references = "\n".join(references[:5])

res = cache['improved_summary'] + "\n\nReferences:\n" + formatted_references

display(Markdown(f"**Topic:** {topic}"))
display(Markdown(res))

**Topic:** Applications of Artificial Intelligence in Education

**Revised Summary:**

**Unlocking the Potential of Education Technology**

In the ever-evolving landscape of education, technology has emerged as a powerful tool to enhance learning outcomes and bridge the gap between students and educators. This summary explores the key concepts and platforms that are revolutionizing the way we learn and teach.

**Online Learning Platforms:**

Online learning platforms have transformed the way we access education, offering a range of features that cater to diverse learning styles and needs. Platforms like Prodigy Education, Achieve Virtual, We Are Teachers, and LearnWorlds provide virtual classrooms, interactive lessons, and personalized learning paths that cater to individual students. For instance, Udemy, Coursera, and edX have democratized access to high-quality educational content, making it possible for anyone to learn new skills and subjects.

However, with the proliferation of online learning platforms, educators and learners face the challenge of sifting through the noise and selecting the most effective tools. To address this, it's essential to consider the following factors:

* **Scalability**: Can the platform accommodate large numbers of students and educators?
* **Customization**: Can the platform be tailored to meet the unique needs of individual students?
* **Interactivity**: Does the platform offer engaging and interactive learning

References:
1. [25 Best Online Learning Platforms for the Virtual Classroom | Prodigy Education](https://www.prodigygame.com/main-en/blog/online-learning-platforms/)
2. [Virtual Learning Platforms and Tools for Teachers and Kids](https://achievevirtual.org/blog/teacher-resources/virtual-learning-platforms-teachers-kids/)
3. [30+ Virtual Learning Platforms and Tools for Teachers and Kids](https://www.weareteachers.com/virtual-learning-platforms/)
4. [Best online learning platform of 2024 | TechRadar](https://www.techradar.com/best/best-online-learning-platforms)
5. [13 Educational Platforms (Updated list 2024) | SC Training](https://training.safetyculture.com/blog/educational-platforms/)